In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TensorFlow Serving 容器为自定义表格模型监控 Vertex AI 模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom_tf_serving.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td> 
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom_tf_serving.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何针对自定义表格模型和自定义部署容器使用Vertex AI模型监控。

了解更多关于[Vertex AI 模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring)。

###目标

在这个笔记本中，您将学习如何使用`Vertex AI Model Monitoring`服务来检测自定义表格模型的输入预测请求中的特征偏差和漂移，使用自定义部署容器。在本教程中，将使用TensorFlow Serving作为自定义部署容器。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Model Monitoring`
- `Vertex AI Prediction`
- `Vertex AI Model`资源
- `Vertex AI Endpoint`资源

执行的步骤包括：

- 下载预训练的自定义表格模型。
- 将预训练模型上传为`Model`资源。
- 使用`TensorFlow Serving`服务二进制部署`Model`资源到`Endpoint`资源。
- 配置`Endpoint`资源进行模型监控。
- 生成用于特征偏差的合成预测请求。
- 等待电子邮件警报通知。
- 生成用于漂移的合成预测请求。
- 等待电子邮件警报通知。

了解更多关于[Vertex AI Model Monitoring介绍](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用一个预训练模型，其中模型工件存储在公共云存储桶中。

该模型基于[博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型的理念是，您的公司有大量描述游戏用户如何与网站互动的日志数据。原始数据包含以下类别的信息：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 关于玩家的信息，比如玩家所在的地理区域
- 行为特征 - 玩家触发特定游戏事件的次数，比如达到新的级别
- 流失倾向 - 这是标签或目标特征，它提供了该玩家可能流失的估计概率，即停止成为活跃玩家。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。

安装

安装执行此笔记本所需的包。

In [ ]:
# Install required packages.
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 google-cloud-bigquery

! gcloud components update --quiet

只有合作：取消下面的单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"

#### 用户邮箱

设置您的用户邮箱地址以接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-user-email]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

### 验证您的 Google Cloud 帐户

根据您的 Jupyter 环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经验证过了。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐户授予云存储权限。

### 关于服务账号和权限的注意事项

**默认情况下不需要任何配置**，如果遇到任何权限相关问题，请确保上述服务账号具有所需的角色：

|服务账号电子邮件|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|计算引擎默认服务账号|Dataflow管理员，Dataflow工作者，存储管理员，BigQuery管理员，Vertex AI用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI平台服务代理|Vertex AI服务代理|

1. 转到https://console.cloud.google.com/iam-admin/iam。
2. 选中“包括提供的Google角色授予”复选框。
3. 找到上述电子邮件。
4. 授予相应的角色。

### 使用不同项目的数据源
- 对于BQ数据源，请授予这两个服务账号“BigQuery数据查看器”角色。
- 对于CSV数据源，请授予这两个服务账号“存储对象查看器”角色。

创建一个云存储桶

创建一个存储桶来存储诸如数据集之类的中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的储存桶尚不存在时才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 初始化 Vertex AI SDK 用于 Python

为您的项目和相应的存储桶初始化 Vertex AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### 创建BigQuery客户端

在本教程中，您将使用与用于训练预训练模型的相同的公共BigQuery表中的数据。您将创建一个客户端接口，随后用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置硬件加速器

您可以为预测（例如，GPU）设置硬件加速器，或选择不使用任何硬件加速器（CPU）。硬件加速器可以降低预测请求的延迟响应时间。在选择硬件加速器时，考虑额外成本与延迟之间的权衡。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像和分配给虚拟机实例（VM）的GPU数量。例如，要使用一个带有4个 Nvidia Tesla K80 GPU 的 GPU 容器映像分配给每个 VM，您应该指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

查看[加速器可用的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，指定`(None, None)`以使用在 CPU 上运行的容器映像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预先构建的容器

为预测设置预先构建的Docker容器镜像。

有关最新列表，请查看[用于预测的预先构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量 `DEPLOY_COMPUTE` 设置为配置预测的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \] 的数字

*备注：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 启用 Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

## 创建一个私有的Docker存储库

您的第一步是在Google Artifact Registry中创建自己的Docker存储库。

1. 运行`gcloud artifacts repositories create`命令，在您的区域创建一个新的Docker存储库，描述为"docker存储库"。

2. 运行`gcloud artifacts repositories list`命令来验证您的存储库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo-unique"

! gcloud services enable artifactregistry.googleapis.com

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置对您的私有存储库的身份验证

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对您所在区域的`Artifact Registry`进行身份验证请求。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

容器（Docker）用于提供的镜像

设置用于提供预测的TensorFlow Serving Docker容器镜像。

1. 从Docker Hub拉取相应的CPU或GPU Docker镜像，用于TF Serving。
2. 为镜像创建一个标签，以在Artifact Registry中注册。
3. 在Artifact Registry中注册该镜像。

了解更多关于[TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker)。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

# Executes in Vertex AI Workbench
if DEPLOY_GPU:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:gpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4-gpu"
else:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:cpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4"

if not IS_COLAB:
    if DEPLOY_GPU:
        ! sudo docker pull tensorflow/serving:2.5.4-gpu
    else:
        ! sudo docker pull tensorflow/serving:2.5.4

    ! docker tag $TF_IMAGE $DEPLOY_IMAGE
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

在谷歌Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE $TF_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $3
docker tag tensorflow/serving $2
docker push $2
kill $(jobs -p)

## Vertex AI模型监控简介

Vertex AI模型监控支持AutoML表格模型和自定义表格模型。您可以监测传入预测请求中的特征的偏斜和漂移检测，或者监测特征归因（可解释AI）在传出预测响应中的偏斜和漂移检测 - 也就是说，对于特征贡献到输出（预测）的分布进行监测。

以下是启用模型监控的基本步骤：

1. 将`Vertex AI`的AutoML或自定义表格模型部署到`Vertex AI Endpoint`。
2. 配置模型监控规范。
3. 将模型监控规范上传到`Vertex AI Endpoint`。
4. 上传或自动生成`输入模式`以进行解析。
5. 对于特征偏斜检测，上传训练数据以自动生成特征分布。
6. 对于特征归因，上传相应的`Vertex AI解释性`规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

当启用模型监控时，采样的传入预测请求将被记录在BigQuery表中。然后，分析记录请求中包含的输入特征值，以检测偏斜或漂移的特征在指定的间隔基础上。您设置一个采样率来监控模型的生产输入的子集，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这被称为输入模式。对于AutoML表格模型，输入模式会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动推导输入模式。另外，您也可以上传输入模式。

对于偏斜检测，监控服务需要训练数据中值的统计分布基线。对于AutoML表格模型，这是自动生成的。对于自定义表格模型，则需将训练数据上传到服务，并让服务自动推导分布。

对于特征归因偏斜和漂移检测，需要为自定义表格模型启用`Vertex AI解释性`。对于AutoML模型，`Vertex AI解释性`会自动启用。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 复制 TensorFlow Serving 的模型文件

*注意:* 对于 TF Serving，MODEL_DIR 必须以一个数字结尾的子文件夹，例如，1。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"
MODEL_DIR = BUCKET_URI + "/model/1"

! gsutil cp -r $MODEL_ARTIFACT_URI $MODEL_DIR

### 将模型工件上传为 `Vertex AI Model` 资源

首先，使用 `upload()` 方法将预训练的自定义表格模型工件作为 `Vertex AI Model` 资源上传，参数如下：

- `display_name`：`Model` 资源的人类可读名称。
- `artifact_uri`：模型工件的云存储位置。
- `serving_container_image`：在将模型部署到 `Vertex AI` 时要使用的服务容器镜像。
- `serving_container_command`：要启动的服务二进制文件（HTTP 服务器）。
- `serving_container_args`：要传递给服务二进制文件的参数。对于 TensorFlow Serving，需要的参数包括：
  - `--model_name`：要分配给模型的人类可读名称。
  - `--model_base_name`：要在容器中存储模型工件的位置。Vertex 服务将变量 $(AIP_STORAGE_URI) 设置为服务在容器中安装模型工件的位置。
  - `--rest_api_port`：用于发送基于 REST 的预测请求的端口。可以是 8080 或 8501（TensorFlow Serving 的默认值）。
  - `--port`：用于发送基于 gRPC 的预测请求的端口。对于 TensorFlow Serving，应将端口设置为 8500。
- `serving_container_health_route`：服务定期ping以验证服务二进制文件是否正在运行的URL。对于 TensorFlow Serving，这将是 /v1/models/\<model_name\>。
- `serving_container_predict_route`：用于将基于 REST 的预测请求路由到服务的 URL。对于 TF Serving，这将是 /v1/models/[model_name]:predict。
- `serving_container_ports`：HTTP 服务器监听请求的端口列表。
- `sync`：是否等待进程完成，或立即返回（异步）。

将模型上传到 Vertex Model 资源会返回一个长时间运行的操作，因为可能需要一些时间。

*注意：* 你需要删除模型路径的末尾数字子文件夹（例如，/1）以进行上传。Vertex 服务将上传模型工件子文件夹上面的父文件夹，这是 TensorFlow Serving 二进制文件所期望的。

*备注：* 当将模型工件上传到 `Vertex AI Model` 资源时，需要指定相应的部署容器镜像。

In [ ]:
MODEL_NAME = "churn"

model = aiplatform.Model.upload(
    display_name="churn",
    artifact_uri=MODEL_DIR[:-2],
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/v1/models/" + MODEL_NAME,
    serving_container_predict_route="/v1/models/" + MODEL_NAME + ":predict",
    serving_container_command=["/usr/bin/tensorflow_model_server"],
    serving_container_args=[
        "--model_name=" + MODEL_NAME,
        "--model_base_path=" + "$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    serving_container_ports=[8080],
    sync=True,
)

print(model)

### 将`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源

接下来，您可以使用`deploy()`方法将您的`Vertex AI Model`资源部署到`Vertex AI Endpoint`资源，使用以下参数：

- `deploy_model_display`：部署模型的人类可读名称。
- `machine_type`：每个VM节点实例的机器类型。
- `min_replica_count`：用于自动扩展的最小节点数量。
- `max_replica_count`：用于自动扩展的最大节点数量。
- `accelerator_type`：每个预配节点的GPU加速器类型（如果有）。
- `accelerator_count`：每个预配节点的GPU加速器数量（如果有）。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

配置监控任务

配置监控任务包括以下规范：

- `alert_config`: 发送监控警报的电子邮件地址。
- `schedule_config`: 分析预测的时间窗口。
- `logging_sampling_strategy`: 采样预测请求的速率。
- `drift_config`: 要监控的特征和漂移阈值。
- `skew_config`: 要监控的特征和偏差阈值。

### 配置警报规范

首先，您可以使用以下设置配置`alerting_config`规范：

- `user_emails`：一个或多个要发送警报的电子邮件列表。
- `enable_logging`：将检测到的异常流向Cloud Logging。默认为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您需要使用以下设置配置`schedule_config`规范：

- `monitor_interval`: 设置模型监控作业的调度间隔，单位为小时。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置抽样规范

接下来，您将使用以下设置配置 `logging_sampling_strategy` 规范：

- `sample_rate`: 随机抽样进行监控预测请求的比例（在 0 到 1 之间）。所选样本将被记录到 BigQuery 表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您可以使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对字典，其中键是要监视漂移的输入特征，值是检测阈值。如果未指定，特征的默认漂移阈值为0.3（30%）。

*注意：*启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置偏斜检测规范

接下来，您可以使用以下设置配置`skew_config`规范：

- `data_source`: 原始训练数据集的数据源。数据源的格式默认为BigQuery表格。否则，必须将设置`data_format`设置为以下值之一。数据的位置必须是Cloud Storage位置。
  - `csv`: 
  - `jsonl`:
  - `tf-record`:
- `skew_thresholds`: 一个键/值对的字典，其中键是监测偏斜的输入特征。值是检测阈值。当未指定时，默认的特征偏斜阈值为0.3（30%）。
- `target_field`: 训练数据集的目标标签

*备注:* 启用偏斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装客观规范

最后，您要使用以下设置组装客观规范 `objective_config`：

- `skew_detection_config`：（可选）偏斜检测配置的规范。
- `drift_detection_config`：（可选）漂移检测配置的规范。
- `explanation_config`：（可选）启用监控特征归因时解释配置的规范。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型的特征输入，这被称为`输入模式`。`输入模式`可以是以下之一
 - 预加载到监控服务中。
 - 在监控服务接收到前1000个预测实例后自动生成。

在本教程中，您将预加载`输入模式`。

#### 创建预定义的输入模式

预定义的`输入模式`被指定为一个YAML文件。在本示例中，您需要检索用于训练数据的BigQuery模式，其中包括特征名称和数据类型，以生成YAML规范。预定义的`输入模式`必须加载到Cloud Storage位置。

了解更多关于[用于解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas).

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控任务

您可以使用`aiplatform.ModelDeploymentMonitoringJob.create()`方法，根据监控规范创建监控任务，其中包括以下参数：

- `display_name`：监控任务的可读名称。
- `project`：项目 ID。
- `region`：地区。
- `endpoint`：要启用监控的 `Vertex AI Endpoint` 的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含 `输入模式` 的 YAML 文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

#### 监控作业的电子邮件通知。

监控作业已启用，将向警报配置中的电子邮件地址发送电子邮件通知。

内容将显示如下：

<blockquote>
您好，Vertex AI客户，

您收到此邮件是因为您正在使用Vertex AI模型监控服务。
此邮件是要通知您，我们已收到您设置漂移或偏斜检测的请求，针对下面列出的预测端点。从现在开始，传入的预测请求将被抽样并记录以供分析。
原始请求和响应将从预测服务中收集并保存在bq://[您的项目ID].model_deployment_monitoring_[端点ID].serving_predict中。
</blockquote>

#### 监控作业状态

在启动`Vertex AI模型监控`作业后，直到计算出`偏斜分布基线`之前，作业会处于`PENDING`状态。监控服务将启动一个批处理作业，从训练数据中生成分布基线。

一旦基线分布生成，监控作业将进入`OFFLINE`状态。每个时间间隔基础上 - 例如，每小时一次，监控作业将进入`RUNNING`状态，同时分析采样数据。完成后，它将返回到`OFFLINE`状态，等待下一次预定的分析。

In [ ]:
jobs = monitoring_job.list(filter="display_name=churn")
job = jobs[0]
print(job.state)

### 基线分布的自动生成

接下来，监控服务会创建一个批处理作业来分析训练数据，生成基线分布。一旦完成，监控服务将开始在指定的间隔进行监控。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(300)

### 生成用于偏斜检测的合成预测请求

接下来，您从BigQuery训练表中提取前1000个实例用于预测请求。您修改数据（合成）以触发预测请求中来自训练分布与服务分布的偏斜检测，具体如下：

- `country`：将所有值设为加拿大。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "country":
            value = "Canada"
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您将使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

一旦监控服务启动了，抽样预测请求将被记录到Cloud Storage中。在下一个监控间隔中，抽样预测将被复制到BigQuery日志表中。一旦条目出现在BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待第一批被记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，您应该看到大约500个条目。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 监控过程中的偏斜检测

特征输入的偏斜检测将在下一个监控间隔中发生。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行过程中，它将分析在此间隔期间从预测中记录的采样表，并将其与基线分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的偏斜，本例中为“国家”，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔上的分析开始到您收到电子邮件警报可能需要多达40分钟。

内容将如下所示

<blockquote>
   你好，Vertex AI 客户，

您收到此邮件是因为您订阅了Vertex AI Model Monitoring服务。
此邮件仅是通知您在部署的模型中检测到了一些异常，并可能需要您的关注。


基本信息：

端点名称：projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监控作业：projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计和异常根路径（Google Cloud存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery命令：SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`


训练预测偏差异常（原始特征）：

异常报告路径（Google Cloud存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关此警报的更多信息，请访问模型监控警报页。

部署的模型ID: <deployed-model-id>

特征名称	异常简要描述	异常详细描述
国家	训练和服务之间的L∞距离较大	L∞距离在训练和服务之间为0.947563（最多六位有效数字），高于阈值0.5。具有最大差异的特征值是：加拿大
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 为漂移检测生成合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例用于预测请求。您修改数据（合成数据）以触发预测请求中的漂移检测，如下所示：

- `cnt_user_engagement`：将值增加4倍。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "cnt_user_engagement":
            value = int(value * 4)
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您将使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在接下来的监控间隔中，抽样预测将被复制到BigQuery日志表中。一旦条目进入BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待首个记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50％，您应该会看到大约1000条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 550:
        break

### 在监控过程中检测数据漂移

特征输入的数据漂移检测将在下一个监控间隔中发生。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态转为“运行”状态。在运行过程中，它将分析此间隔内预测的已记录样本表，并将其与先前监控间隔的分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的数据漂移，本例中为“cnt_user_engagement”，并在下一个间隔前将监控作业置为“离线”状态。

#### 等待监控间隔

从分析监控间隔开始到收到电子邮件警报可能需要高达40分钟。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

删除监控任务

您可以使用 `delete()` 方法来删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署并删除 `Vertex AI Endpoint` 资源

您可以使用 `delete()` 方法删除您的 `Vertex AI Endpoint` 资源。在删除之前，必须先取消部署到您的 `Vertex AI Endpoint` 资源的任何模型。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}

! gcloud artifacts repositories delete {PRIVATE_REPO} --location={REGION} --quiet 